In [1]:
import os

list_file = sorted(list(os.walk('./file/2.part'))[0][2])
while '.DS_Store' in list_file:
    list_file.remove('.DS_Store')
while '0.null.txt' in list_file:
    list_file.remove('0.null.txt')

list_file

['crawler_1.xlsx', 'crawler_2.xlsx']

In [2]:
file = list_file[0]

file

'crawler_1.xlsx'

In [3]:
import pandas as pd

output_correct, output_error = pd.DataFrame(), pd.DataFrame()

In [4]:
input_ = pd.read_excel(f'./file/2.part/{file}',
                       header=0,
                       dtype=str).fillna('')

input_['No'] = input_['No'].astype(int)

print(f'总数量：{len(input_)}')
input_['JOIN_MPNTCPC'].tolist()

总数量：10


['MELLING;30250S;5756;8304960',
 'MELLING;30250SX;5756;11158045',
 'MELLING;30420S;5756;8304968',
 'MELLING;30430S;5756;8304972',
 'MELLING;30460S;5756;8304976',
 'MELLING;30460SA;5756;9658980',
 'MELLING;30460SB;5756;9658984',
 'MELLING;30470S;5756;8304980',
 'MELLING;30470SX;5756;10110000',
 'MELLING;30490S;5756;8304984']

In [5]:
a = 0

a

0

In [6]:
crawler_status = 'error'

crawler_status

'error'

In [7]:
import requests
import json

import sys
sys.path.append('../00.Tools')
from crawler_configuration import get_header, get_proxy

payload = {'partData': {'listing_data_essential': {'parttype': input_.loc[a, 'Type Code'],
                                                   'partkey': input_.loc[a, 'Part Code']},
                        'listing_data_supplemental': {'partnumber': input_.loc[a, 'Part Number'],
                                                      'catalogname': input_.loc[a, 'Manufacturer']}}}

data = {'func': 'getbuyersguide',
        'payload': json.dumps(payload),
        'api_json_request': '1'}

b = 0
while True:
    b += 1
    print(b)
    try:
        check_url = input_.loc[a, 'Url']
        
        resp = requests.post('https://www.rockauto.com/catalog/catalogapi.php',
                             data=data,
                             headers=get_header(),
                             proxies=get_proxy(),
                             timeout=(10, 10))

        if resp.status_code == 200:
            dict_ = resp.json()
    
            if 'redirect_to_url' not in dict_ and dict_['buyersguidepieces']['title'].startswith('Buyer\'s Guide :') and 'collected_javascript' in dict_:
                break
    except KeyboardInterrupt:
        break
    except:
        continue

resp

1


<Response [200]>

In [8]:
from bs4 import BeautifulSoup
from lxml import etree

soup = BeautifulSoup(dict_['buyersguidepieces']['body'], 'lxml')
html = etree.HTML(str(soup))

with open('./vehicle.html', 'w', encoding='UTF-8') as file:
    file.write(str(soup))

print(soup.prettify())

<html>
 <body>
  <div class="buyersguide-nested">
   <div style="padding: .5em;">
    <table class="nobmp" style="width: 100%;">
     <tr>
      <td class="altrow-a-0">
       MAZDA
      </td>
      <td class="altrow-a-0">
       3
      </td>
      <td class="altrow-a-0">
       2004-2007
      </td>
     </tr>
     <tr>
      <td class="altrow-a-1">
       MAZDA
      </td>
      <td class="altrow-a-1">
       5
      </td>
      <td class="altrow-a-1">
       2006-2007
      </td>
     </tr>
     <tr>
      <td class="altrow-a-0">
       MAZDA
      </td>
      <td class="altrow-a-0">
       6
      </td>
      <td class="altrow-a-0">
       2003-2007
      </td>
     </tr>
    </table>
    <div>
     Please refer to catalog for application details.
    </div>
   </div>
  </div>
 </body>
</html>



In [9]:
len(html.xpath('//p/text()')) != 0 and html.xpath('//p/text()')[0] == 'No applications found.'

False

In [10]:
list_tr = html.xpath('//tr')

list_tr

[<Element tr at 0x11f6b7b80>,
 <Element tr at 0x11f6b6940>,
 <Element tr at 0x11f6b6680>]

In [11]:
len(list_tr) == 0

False

In [12]:
df_temp = pd.DataFrame()
for i, tr in enumerate(list_tr):
    list_text = [text.strip() for text in tr.xpath('./td/text()')]

    if len(list_text) == 3:
        if '-' in list_text[2]:
            [year_begin, year_end] = list_text[2].split('-')
            for year in range(int(year_begin), int(year_end) + 1):
                df_temp_temp = pd.DataFrame([{'No': input_.loc[a, 'No'],
                                              'Part Number': input_.loc[a, 'Part Number'],
                                              'JOIN_MPNTCPC': input_.loc[a, 'JOIN_MPNTCPC'],
                                              'Vehicle No': i + 1,
                                              'Make': list_text[0],
                                              'Model': list_text[1],
                                              'Year': year}])
                df_temp = pd.concat([df_temp, df_temp_temp], ignore_index=True).fillna('')
        else:
            df_temp_temp = pd.DataFrame([{'No': input_.loc[a, 'No'],
                                          'Part Number': input_.loc[a, 'Part Number'],
                                          'JOIN_MPNTCPC': input_.loc[a, 'JOIN_MPNTCPC'],
                                          'Vehicle No': i + 1,
                                          'Make': list_text[0],
                                          'Model': list_text[1],
                                          'Year': int(list_text[2])}])
            df_temp = pd.concat([df_temp, df_temp_temp], ignore_index=True).fillna('')
    elif len(list_text) == 2:
        if '-' in list_text[1]:
            [year_begin, year_end] = list_text[1].split('-')
            for year in range(int(year_begin), int(year_end) + 1):
                df_temp_temp = pd.DataFrame([{'No': input_.loc[a, 'No'],
                                              'Part Number': input_.loc[a, 'Part Number'],
                                              'JOIN_MPNTCPC': input_.loc[a, 'JOIN_MPNTCPC'],
                                              'Vehicle No': i + 1,
                                              'Make': list_text[0],
                                              'Model': '',
                                              'Year': year}])
                df_temp = pd.concat([df_temp, df_temp_temp], ignore_index=True).fillna('')
        else:
            df_temp_temp = pd.DataFrame([{'No': input_.loc[a, 'No'],
                                          'Part Number': input_.loc[a, 'Part Number'],
                                          'JOIN_MPNTCPC': input_.loc[a, 'JOIN_MPNTCPC'],
                                          'Vehicle No': i + 1,
                                          'Make': list_text[0],
                                          'Model': '',
                                          'Year': int(list_text[1])}])
            df_temp = pd.concat([df_temp, df_temp_temp], ignore_index=True).fillna('')
    else:
        print('fuck')

output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')

output_correct

,No,Part Number,JOIN_MPNTCPC,Vehicle No,Make,Model,Year
0,1,30250S,MELLING;30250S;5756;8304960,1,MAZDA,3,2004
1,1,30250S,MELLING;30250S;5756;8304960,1,MAZDA,3,2005
2,1,30250S,MELLING;30250S;5756;8304960,1,MAZDA,3,2006
3,1,30250S,MELLING;30250S;5756;8304960,1,MAZDA,3,2007
4,1,30250S,MELLING;30250S;5756;8304960,2,MAZDA,5,2006
5,1,30250S,MELLING;30250S;5756;8304960,2,MAZDA,5,2007
6,1,30250S,MELLING;30250S;5756;8304960,3,MAZDA,6,2003
7,1,30250S,MELLING;30250S;5756;8304960,3,MAZDA,6,2004
8,1,30250S,MELLING;30250S;5756;8304960,3,MAZDA,6,2005
9,1,30250S,MELLING;30250S;5756;8304960,3,MAZDA,6,2006


In [13]:
crawler_status = 'ok'

crawler_status

'ok'

In [14]:
if crawler_status == 'error':
    df_temp = pd.DataFrame([input_.iloc[a]]).reset_index(drop=True)
    df_temp.loc[0, 'Check_Url'] = check_url
    
    output_error = pd.concat([output_error, df_temp], ignore_index=True).fillna('')

output_error

""


In [15]:
if not output_correct.empty:
    output_correct = output_correct.sort_values(by=['No', 'Vehicle No', 'Year'],
                                                ascending=[True, True, False],
                                                ignore_index=True)
    output_correct.to_excel('./test_vehicle.xlsx', index=False)

output_correct

,No,Part Number,JOIN_MPNTCPC,Vehicle No,Make,Model,Year
0,1,30250S,MELLING;30250S;5756;8304960,1,MAZDA,3,2007
1,1,30250S,MELLING;30250S;5756;8304960,1,MAZDA,3,2006
2,1,30250S,MELLING;30250S;5756;8304960,1,MAZDA,3,2005
3,1,30250S,MELLING;30250S;5756;8304960,1,MAZDA,3,2004
4,1,30250S,MELLING;30250S;5756;8304960,2,MAZDA,5,2007
5,1,30250S,MELLING;30250S;5756;8304960,2,MAZDA,5,2006
6,1,30250S,MELLING;30250S;5756;8304960,3,MAZDA,6,2007
7,1,30250S,MELLING;30250S;5756;8304960,3,MAZDA,6,2006
8,1,30250S,MELLING;30250S;5756;8304960,3,MAZDA,6,2005
9,1,30250S,MELLING;30250S;5756;8304960,3,MAZDA,6,2004


In [16]:
if not output_error.empty:
    output_error = output_error.sort_values(by=['No'],
                                            ascending=[True],
                                            ignore_index=True)
    output_error.to_excel('./test_vehicle_error.xlsx', index=False)

output_error

""
